# Global Food Prices

Get dataset

In [93]:
import os
from download import download
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

url_2 = "https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv"
regional_file_name = "regions.csv"

url = "http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_05-12-2017.csv"
file_name = 'WFPVAM_FoodPrices_05-12-2017.csv'

def get_dataset(url, file_name):
    if not os.path.isfile(file_name):
        download(url, file_name)
    
    return pd.read_csv(file_name, encoding='latin-1')

df = get_dataset(url, file_name)

region_df = get_dataset(url_2, regional_file_name)

In [98]:
# Pre-process
region_df.rename(columns={'name': 'adm0_name'}, inplace=True)
new_regions = region_df.loc[:, ['adm0_name', 'sub-region']]

df_regions = pd.merge(df, new_regions, on='adm0_name', how='left')

df = df_regions.copy()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,sub-region
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.00,WFP,Southern Asia
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.00,WFP,Southern Asia
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.00,WFP,Southern Asia
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.00,WFP,Southern Asia
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.00,WFP,Southern Asia
5,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,6,2014,50.00,WFP,Southern Asia
6,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,7,2014,50.00,WFP,Southern Asia
7,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,8,2014,50.00,WFP,Southern Asia
8,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,9,2014,50.00,WFP,Southern Asia
9,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,10,2014,50.00,WFP,Southern Asia


In [103]:
# Get average prices
print(df.columns.values.tolist())
# avg_prices = df.loc[df['mp_prices']]

df['datetime'] = pd.to_datetime(df.mp_year*10000+df.mp_month*100+1, format='%Y%m%d')

avg_price = df.groupby(['sub-region', 'cm_name', 'datetime'])['mp_price']



df_means = avg_price.mean().reset_index()

['adm0_id', 'adm0_name', 'adm1_id', 'adm1_name', 'mkt_id', 'mkt_name', 'cm_id', 'cm_name', 'cur_id', 'cur_name', 'pt_id', 'pt_name', 'um_id', 'um_name', 'mp_month', 'mp_year', 'mp_price', 'mp_commoditysource', 'sub-region', 'datetime']


In [104]:
# plt.plot(df_means['datetime'], df_means['mp_price'], '-')

# Make a graph per product. Each line represents a country (or region for later).


from bokeh.io import output_file, show, save
# output_notebook()

from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure

In [105]:
from bokeh.palettes import Dark2_5 as palette
import itertools
from bokeh.layouts import column

b = df_means.groupby(['cm_name'])

figures = []

idx = 0
for group, row in b:
    color_idx = 0
    country_group = row.groupby(['sub-region'])
    p = figure(plot_width=800, plot_height=300, title=group, x_axis_type='datetime')
    for group2, row2 in country_group:
        datetime = []
        prices = []
        for i, data in row2.iterrows():
            datetime.append(data['datetime'])
            prices.append(data['mp_price'])
        p.line(datetime, prices, line_width=4, legend=data['sub-region'], color=palette[color_idx % 5])
        color_idx += 1
    figures.append(p)
    idx += 1
    if idx > 20:
        break


save(column(figures))

'/var/folders/tt/h7nt6h7x2rd7txpbf_8ry5940000gn/T/tmpknq5rx8d.html'